# TFIDF for large_set

In [1]:
from pyspark import SparkConf, SparkContext
import os, re, time, math
from pyspark.sql.session import SparkSession

In [2]:
#path = r'/root/workshop32/dataset/'
# ensure you copy all the datafiles into this position in all the spark clusters !
# copy by scp: scp -r /root/workshop32 root@workshop32slave2:/root/

path = r'hdfs://workshop32master:9000/workshop32/dataset/'

In [3]:
SparkContext.setSystemProperty('spark.executor.memory', '8g')
SparkContext.setSystemProperty('spark.driver.memory', '8g')
spark = SparkSession.builder.master('spark://workshop32master:7077').appName('MyTf-Idf').getOrCreate()
# use spark cluster
    
sc = spark.sparkContext
#sc = SparkContext.getOrCreate(SparkConf())
# sc._conf.set('spark.executor.memory', '12g')
# sc._conf.set('spark.driver.memory', '12g')
# sc._conf.set('spark.driver.maxResultsSize', '0')

sc

21/12/15 08:01:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=spark://workshop32master:7077 appName=MyTf-Idf>

In [4]:
# sparkIndex
data = sc.wholeTextFiles(path+'large_set')

numFiles = data.count()

timeStart = time.time()
wordcount = data.flatMap(lambda x: [((os.path.basename(x[0]), i), 1) \
        for i in re.split('\\W', x[1])]) \
        .reduceByKey(lambda a, b: a + b)

tf = wordcount.map(lambda x: (x[0][1].lower(),(x[0][0],x[1])))
tf_list = tf.collect()
timeEnd = time.time()

print("Execution Time: ",(timeEnd - timeStart))

Execution Time:  630.2132616043091


In [5]:
tf_list[:10]

[('2002', ('5002.txt', 1)),
 ('student', ('5002.txt', 27)),
 ('vas', ('5002.txt', 1)),
 ('consciousness', ('5002.txt', 2)),
 ('verses', ('5002.txt', 4)),
 ('filing', ('5002.txt', 1)),
 ('nevertheless', ('5002.txt', 6)),
 ('capable', ('5002.txt', 1)),
 ('cloud', ('5002.txt', 2)),
 ('chuckled', ('5002.txt', 2))]

In [6]:
timeStart = time.time()
tf2 = tf.map(lambda x: (x[0],(x[1][0], 1+math.log10((x[1][1]) ))))

idf = wordcount.map(lambda x: (x[0][1], (x[0][0], x[1], 1)))\
        .map(lambda x: (x[0], x[1][2]))\
        .reduceByKey(lambda x, y: x + y)\
        .map(lambda x: (x[0], math.log10(numFiles / (x[1]))))

tfidf = tf2.join(idf)
tfidf = tfidf.map(lambda x: (x[1][0][0], (x[0], x[1][0][1] * x[1][1]))).sortByKey()
index = tfidf.map(lambda x:(x[1][0], (x[0], x[1][1]))).filter(lambda x: x[0]!='')
index = index.sortByKey()
lines = index.map(lambda x: (x[0], x[1][0], x[1][1]))
lines_list = lines.collect()

ranking_dict = {}
for i in lines_list:
    ranking_dict[i[0]] = ranking_dict.get(i[0],[]) + [(i[1],i[2])]
    
for i in ranking_dict.items():
    i[1].sort(key = lambda x:x[1], reverse = True)
    ranking_dict[i[0]] = i[1]

with open("tfidf_index_large.txt", "w") as f:
    for i in ranking_dict.items():
        # to write in a specific way <word>:[]
        f.write(i[0]+":[")
        for j in range(len(i[1])):
            f.write(str(i[1][j][0]))
            if(j == len(i[1])-1):
                f.write('('+str(i[1][j][1])+")]\n")
            else:
                f.write('('+str(i[1][j][1])+"),")
        
timeEnd = time.time()

print("Execution Time: ",(timeEnd - timeStart))

Execution Time:  367.4779131412506


In [7]:
# FOR DEBUG
#print([i for i in ranking_dict.items()][66654])

('chattily', [('2579.txt', 2.7849737099544005), ('1718.txt', 2.7849737099544005), ('2233.txt', 2.7849737099544005)])


In [13]:
# # code for retreiving the tfidf index from tfidf_index.txt

# tfidf_open = []

# with open("tfidf_index.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         line = line.strip('\n')
#         tfidf_open.append(eval(line))

# tfidf_open[:10]

In [9]:
# # FOR DEBUG: getting a slice of tfidf_index
# with open("tfidf_index_slice.txt", "w") as f:
#     for j in range(66666, 67666):
#         i = tfidf_open[j]
#         f.write(str(i)+'\n')